# Installation

In [1]:
!pip install -U langgraph langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.5 MB/s eta 0:00:00


In [2]:
!pip show langgraph langchain_openai

Name: langgraph
Version: 0.3.11
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk
Required-by: 
---
Name: langchain-openai
Version: 0.3.8
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


# Part 1: Build a Basic Chatbot

In [ ]:
################################
# Part1: Build a Basic Chatbot #
################################

# Start by creating a StateGraph, A State object defines the structure
# of our chabot as "state machine", We'll add `nodes` to represent the llm
# and functions our chatbot can call and `edges` to specifiy how the bot
# should transition between these functions
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    # Message have the type "list", The "add_messages" function
    # in the annotation defines how this state should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    message: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Next, add a "chatbot" node, Nodes represent units of work,
# They are typically regular python funcitons
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="")

def chatbot(state: State):
    # Notice how the chatbot node function takes the current `State` as
    # input adn returns a dictionary containing an updated `messages` list under the key "messages".
    # This is the basic pattern for all LangGraph node functions
    return {"messages": [llm.invokes(state["messages"])]}

# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# The node is used
graph_builder.add_node("chatbot", chatbot)

# Next, add an `entry` point.
# This tells our graph where to start its work each time we run it.
graph_builder.add_edge(START, "chat_bot")
# Similarly, Set a finish point.
# This instructs the graph "any time this node is run, you can exit."
graph_builder.add_edge("chatbot", END)
# Finally, we'll want to be able to run our graph, call "compile()" on the graph builder.
# This creates a "CompiledGraph" we can use invoke on our state
graph = graph_builder.compile()


# You can visualize the graph using the `get_graph` method and one of the "draw"
# methods, like `draw_ascii` or `draw_png`. the `draw` methods each require additional dependencies
from IPython.display import Image, display
try:
    display(Image(graph.get_graph).draw_mermaid_png())
except Exception:
    pass

# Now let's run the chatbot!!!
def stream_graph_updates(user_input: str):
    messages = {"messages": [{"role": "user", "content": user_input}]}
    for event in graph.stream(messages):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User:")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input=user_input)
    except:
        pass

# Congratulations!!! You're built your first chatbot using LangGraph.
# This bot can engage in basic conversation by taking user input and generating responses using an LLM
# However, you may have noticed that the bot's knowledge is limited to what's in its training date. In the
# next part, we'll add a web search tool to expand the bot's knowledge and make it more capable.

# Blow is the full code for this section for your reference:
from typing import Annotated
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class State(TypedDict):
    message: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm = ChatAnthropic(model="")

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

# Part 2: Enhancing the Chatbot with Tools

In [ ]:
###########################################
# Part2: Enhancing the Chatbot with Tools #
###########################################

# To handle queries our chatbot can't answer "from memory",
# we'll integrate a web search tool. Our bot can use this tool
# to find relevant information and provide better responses.

# Define the tool:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults(max_result=2)
tools = [tool]
tool.invoke("What's a `node` in LangGraph?") # The results are pages summaries

# Next, we'wll start defining our graph.
# The following is all the same as Part1, except we have added `bind_tools` on our LLM.
# This lets the LLM konw the correct JSON format to use if it wawnts to use our search engine.
from typing import Annotated
from langchain_anthropic import ChatAnthropic
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    message: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm = ChatAnthropic(model="")
# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
#...

# Next we need to create a function to actually run the tools if they are called.
# We'll do this by adding the tools to a new node.
# Below we implement a `BasicToolNode` that checks the most recent messages in the state
# and calls tools if the messages contains `tool_calls`. It relies on  the LLM's `tool_calling`
# support, which is available in Anthropic, OpenAI...

# We will later replace this with LangGraph's prebuilt `ToolNode` to speed
# things up, but building it ourselves first is instructive.
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """ A node that runs the tools requested in the last AIMessages """
    def __init__(self, tools:list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tool}

    def __call__(self, inputs:dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No messages found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"].invoke(tool_call["args"])]
        outputs.append(
            ToolMessage(
                tool_call_id=tool_call["id"],
                name=tool["name"],
                content=json.dump(tool_result),
            )
        )
        return {"messages": outputs}

tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

# Below, call define a router function called `route_tools`,
# that checks for tool_calls in the chatbot's output.
# Provide this function to the graph by calling `add_conditonal_edges`,
# which tells the graph that whenever the `chatbot` node completes to check this function to see where to go next.

# The condition will route to `tools` if tool calls are present and `END` if not.
# Later, we will replace this with the prebuilt `tools_condition` to be more concise,
# but implementing it ourselves first makes things more clear.
def route_tools(state: State):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    # 1.如果 state 是一个列表，那么 ai_message 被设置为列表的最后一个元素
    if isinstance(state, list):
        ai_message = state[-1]
    # 2.如果 state 是一个字典，并且包含键 "messages"，那么 ai_message 被设置为 state["messages"] 列表的最后一个元素
    elif message := state.get("messages", []):
        ai_message = message[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    # 3.如果 ai_message 对象有 tool_calls 属性，并且 tool_calls 列表不为空，则返回字符串 "tools"
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_call) > 0:
        return "tools"
    return END

# The `tools_condition` function returns "tools" if the chatbot asks to use a tool
# and "END" if it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The folloing dictionary lets you tell the graph
    # to interpret the condition's outputs as a  specific node
    # It defaults to the identity function, but if you want to use a node named something else apart from "tools",
    # you can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", END: END}
)

# Any time a tool is called, we return to the chatbot to decide the next step
# Note that we don't need to explicity set a `finish_point` this time,
# because our graph already has a way to finish!
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

# Now we can ask the bot questions outside its training data.
while True:
    try:
        user_input = input("User:")
        if user_input.lower() in ["quit", "eixt", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input=user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

# Congrats! You've created a conversational agent in langgraph that can use a search engine to retrieve
# updated information when needed. Now it can handle a wider range of user queries.

# Our chatbot still can't remember past interactions on its own, limiting its ability to have coherent,
# multi-turn conversation. In the next part, we'll add memory to address this.

# The full code for the graph we've created in this section is reproduced below, replacing our `BasicToolNode`
# for the prebuilt `ToolNode`, and our `route_tools` condition with the prebuilt `tool_condition`
from typing import Annotated
from langchain_anthropic import ChatAnthropic
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    message: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearchResults(max_result=2)
tools = [tool]
llm = ChatAnthropic(model="")
llm_with_tools = llm.bind_tools[tools]

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# Any time a tool is called,
# We return to the chatbot to decide the next step.
# !!!Note that we don't need to explicity set a `finish_point` this time,
# because our graph already has a way to finish!!!
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

# Part 3: Adding Memory to the Chatbot

In [ ]:
########################################
# Part 3: Adding Memory to the Chatbot #
########################################
# Our chatbot can now use tools to answer user question, buts it doesn'ts remembers
# the context of previouss interactions. This limits its ability to have coherent, multi-turn conversations.

# LangGraph solves this problem through persistent checkpointing. If you provide a `checkpointer` when compiling
# the graph and `thread_id` when calling your graph, LangGraph automatically saves the sate after each step.
# When you invoke the graph again using the same `thread_id`, the graph loads its saved state,
# allowing the chatbot to pick up where it left off.

# We will see later that checkpointing is much more powerful than simple chat memroy - it lets
# you save and resume complex state at any time for error recovery, human-in-loop workflows,
# time travel interacions, and more. But before we get too ahead of ourselves, let's add checkpointing
# to enable multi-turn conversations.

# Notice: we're using an in-memory checkpointer. This is convenient for our tutorial(it saves it all in-memory).
# In a production application, you would like change this to use `SqliteSaver` or `PostgresSaver` and connect to
# your own DB.

# The following is all copied from Part 2.
from typing import Annotated
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Part 4: Human-in-the-loop

# Part 5: Customizing State

# Part 6: Time Travel